# 한글 text mining


In [ ]:
import sys
print(sys.version)

# 1. 패키지 설치

### http://konlpy.org 에 접속하여 java 등 필요한 프로그램을 설치한다.



### 참고 사이트

http://www.lifebloom.biz/2017/07/04/python%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EB%A7%88%EC%9D%B4%EB%8B%9D-1-%EB%AA%A8%EB%93%88%EC%84%A4%EC%B9%98/

### pip install beautifulsoup4

### pip install pygame

### pip install simplejson

### pip install -U pytagcloud



<br><br>
# 2. 데이터 수집 (Web Scraping)

* 텍스트를 분석 역시 일반적인 데이터 분석처럼 데이터 수집 -> 데이터 전처리 -> EDA -> 데이터 분석 -> knowledge 도출의 단계를 거칩니다. 
* 하지만 unstructured data인 관계로 데이터의 수집과 전처리 과정에서 큰 차이가 나타납니다.
* movie.daum.net의 영화 리뷰 댓글을 이용해서 데이터를 수집 하기로 하자.


* 기본적으로 윈도우 환경의 Python은 ANSI 형식을 따르기 때문에 한글이 깨지는 경우가 발생한다. 
* 그래서 Jupyter notebook에서는 맨 처음 code 셀 에   아래와 같이 입력하여 기본 인코딩 형식을 utf-8로 바꾸어야 텍스트를 불러왔을때 깨지지 않게 된다.

#-*- coding: utf-8 -*-

<br>
* 필요한 모듈 설치

In [ ]:
from urllib.request import urlopen

from bs4 import BeautifulSoup
import pandas as pd

<br>
* 먼저 movie.daum.net의 리뷰 댓글들중 한 페이지를 불러오자.

<br>
* urlopen을 이용해 url주소의 리뷰 데이터를 객체에 저장하자.

<br>
* html.parser를 사용하여 HTML 문을 텍스트 형태로 변환하자.

<br>
* 페이지 2의 리뷰들을 모두 추출했다.
* 이제, for문을 통해 url을 리스트 형식으로 만들어 여러 페이지의 데이터를 추출하자.

In [ ]:
review_list=[]
for n in range(2):
    #url = 'http://movie.daum.net/moviedb/grade?movieId=110929&type=netizen&page={}'.format(n+1)
    url = 'http://movie.daum.net/moviedb/grade?movieId=97728&type=netizen&page={}'.format(n+1)
    
    webpage = urlopen(url)
    source = BeautifulSoup(webpage,'html.parser',from_encoding='utf-8')    
    review_info= source.findAll('div', {'class': 'review_info'})

    for review_long in review_info:       
        glade = review_long.find('em', {'class': 'emph_grade'}) 
        glade2= glade.get_text().strip()
        
        review = review_long.find('p', {'class': 'desc_review'})   
        comment=review.get_text().strip().replace('\n',"").replace('\t',"").replace('\r',"")
        
        if comment=='' or comment=='.':
            pass
        elif int(glade2) > 0 and int(glade2) < 10:
            review_list.append((int(glade2),comment))
        

print(review_list)
print(type(review_list[0]))
reviews_pd=pd.DataFrame(review_list, columns=['rate','comment'])

In [ ]:
reviews_pd.head()

In [ ]:
with open('reviews.csv','w') as f:
    reviews_pd.to_csv(f, index=False)
f.close()

<br><br>
# 3. 텍스트 전처리 (Preprocessing)

<br>
* 앞에서 저장했던 review.csv 화일을 읽어 들이자.

In [ ]:

reviews2_pd=pd.read_csv('reviews.csv',encoding='cp949')
#reviews_pd=pd.read_csv('reviews.csv',encoding='cp949' )
#reviews_pd=pd.read_csv('review.csv',encode='utf-8' )

In [ ]:
print(reviews2_pd)

In [ ]:
print(reviews2_pd["comment"])

In [ ]:
comments=reviews2_pd["comment"]

# 아래 셀은 테스트 셋을 읽기 위함 임.

reviews2_pd=pd.read_csv('test.csv',encoding='cp949')
comments=reviews2_pd["comment"]

### 감성분석

In [ ]:
import csv
with open("dic/polarity.txt", "r", encoding='UTF8') as polarity:
    dic = []
    cr = csv.reader(polarity)
    for row in cr:
        dic.append(row)

In [ ]:
df=pd.DataFrame(dic[1:],columns=dic[0])

In [ ]:
#import socket, sys, json
from konlpy.tag import Kkma
from konlpy.utils import pprint


In [ ]:
kkma = Kkma()

In [ ]:
pprint(kkma.pos("가까이"))


In [ ]:
polarity_dict={}
for ele in dic[1:]:
    #print(ele)
    #print(ele[0])
    #print(ele[1:])
    polarity_dict[ele[0]]=ele[1:]

In [ ]:
print(polarity_dict)

In [ ]:
reviews2_pd

In [ ]:
comments

In [ ]:
sentiments=[]
n=0
for text in comments:
    print(n)
    print(text)
    n=n+1
    kkma = Kkma()
    #pprint(kkma.pos(text))
    tokens = kkma.pos(text)
    score = []
    neg = 0
    neut = 0
    none = 0
    pos = 0
    i=0 # current location  
    m=0
    #print("tokens",tokens)
    token_pos=[]
    for token in tokens:
        token_pos.append("/".join(token))

    #print(token_pos)    
    num=len(token_pos)  
    #print(num)

    neg=0
    neut=0
    non=0
    pos=0

    while True:
        #print(i)
        if i+3 <= num:
            try: 
                #print(";".join(token_pos[i:i+3]))
                x=polarity_dict[";".join(token_pos[i:i+3])]
                m=3
                #print("AAAA")
            except:
                try:
                    #print(";".join(token_pos[i:i+2]))
                    x=polarity_dict[";".join(token_pos[i:i+2])]
                    m=2
                    #print("BBB")
                except:
                    try:
                        x=polarity_dict[token_pos[i]]
                        m=1
                        #print("CCC")
                    except:
                        m=0
                        #print("DDD")
        elif i+2 <= num:
            try: 
                #print(";".join(token_pos[i:i+2]))
                x=polarity_dict[";".join(token_pos[i:i+2])]
                m=2
                #print("EEE")
            except:
                try:
                    #print(";".join(token_pos[i]))
                    x=polarity_dict[";".join(token_pos[i])]
                    m=1
                    #print("FFF")
                except:
                    m=0
                    #print("GGG")

        elif i+1 <= num:
            try:
                #print(";".join(token_pos[i]))
                x=polarity_dict[";".join(token_pos[i])]
                m=1
                #print("HHH")
            except:
                m=0
                #print("III")
        else:
            print("STOP !!!!")
            break

        if m > 0:
            item=polarity_dict[";".join(token_pos[i: i+m])]
            #print(item)
            neg += float(item[2]) * float(item[7])
            neut += float(item[3]) * float(item[7])
            none += float(item[4]) * float(item[7])
            pos += float(item[5]) * float(item[7])
            i=i+m
        else:
            i += 1

        #print(neg,neut,non,pos)    
        #print("m=",m)   
        #print("i=",i)




    #print(neg,neut,non,pos)    
    #print("m=",m)   
    score=[]
    total = neg + neut + none + pos
    if total == 0:
        total = 1
    score.append(round(pos / total,2))
    score.append(round((neut+none)/total,2))
    score.append(round(neg / total,2))
    print("*********************************")
    print(text)
    print("긍정 :",  score[0] ,"중립", score[1], "부정 :" , score[2])
    if score[0] >= 0.7:
        print("Positive")
    elif score[0] < 0.4:
        print("Negative")
    else:
        print("Neutral")
    sentiments.append((score[0],score[1],score[2]))
print(sentiments)

In [ ]:


reviews2_pd.insert(loc=1, column='sentiment2', value=sentiments)

In [ ]:
reviews2_pd

In [ ]:
sentiment_pred=[]
for pos, neu, neg in reviews2_pd["sentiment2"]:
    max_val=max(pos,neu,neg)
    if max_val==pos:
        s=2
    elif max_val==neu:
        s=1
    else:
        s=0
    sentiment_pred.append(s)

print(sentiment_pred)
    

In [ ]:
reviews2_pd['sentiment_pred']=sentiment_pred

In [ ]:
reviews2_pd

## 결과를 csv 화일에 쓰기

In [ ]:
reviews2_pd['sentiment_pred'].to_csv('result_1.csv', index=False)

<br>
<br>
# 비교 검토

In [ ]:
sentiment_real = ((reviews2_pd['rate']-1)//3)

In [ ]:
sentiment_real

## 감성 예측 정확도

In [ ]:
sum(sentiment_real==sentiment_pred)

## 감성 예측 에러

In [ ]:
sum(abs(sentiment_real-sentiment_pred))